# LangChain Chains

Aqui nos enfocaremos en las cadenas o Chains que son muy importantes en la libreria LangChain y por ello estan en su nombre.

Las Chains nos permiten encadenar la salida de una consulta a un LLM con la entrada a otra consulta y asi conseguir mejores resultados y mas complejos.

Una cadena se compone de un modelo,un prompt de entrada y su respectiva salida. Veamos algunos ejemplos:

In [ ]:
!pip install openai --upgrade --quiet
!pip install langchain --upgrade --quiet

In [ ]:
import openai

api_key="INGRESA AQUI TU TOKEN DE OPENAI"

In [ ]:
#@title [Opcional] cargar el token desde gdrive para que no se vea al dar clases {display-mode:"form"}
if api_key == "INGRESA AQUI TU TOKEN DE OPENAI":
    from google.colab import drive
    drive.mount('/content/drive')
    with open('drive/MyDrive/Tokens/openai.txt','r') as f:
        token = f.read()
        openai.api_key = token

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(openai_api_key=token, temperature=0.0, model="gpt-3.5-turbo")

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "Queen Size Sheet Set"
chain.run(product)

## Simple Sequential Chain

En este ejemplo como su nombre lo indica cada cadena es simple y esto significa que solo tiene una entrada y una salida y es secuencial porque la la salida de una cadena sera la entrada de otra.

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(openai_api_key=token, temperature=0.0, model="gpt-3.5-turbo")

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# chain 1
chain_1 = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_2 = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
cadena_simple_secuencial = SimpleSequentialChain(
    chains=[chain_1, chain_2],
    verbose=True
)

In [ ]:
cadena_simple_secuencial.run(product)

## Sequential Chain

En este caso podemos hacer que una cadena tenga como entrada la salida de mas de una cadena. Por ello ahora utilizaremos el atributo **output_key** para darle un nombre a cada salida y luego poder utilizarla mas tarde como entrada en otra cadena.

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(openai_api_key=token, temperature=0.0, model="gpt-3.5-turbo")

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [ ]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [ ]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
review = "Esta comida estuvo deliciosa"
overall_chain(review)

## Router Chain

### Esto ha sido deprecado. Actualizar el tutorial a Router class
- https://python.langchain.com/docs/modules/chains/foundational/router

En este caso podemos elegir que cadena queremos utilizar de acuerdo al tipo de salida de la cadena anterior.

## Referencias:
Esta unidad esta basada en el [curso de Andrew Ng de Deeplearning.ai](https://www.deeplearning.ai/short-courses/langchain-for-llm-application-development/)

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)